# Regularized linear models 

- Ridge regression
- LASSO
- ElasticNet

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("../data/Advertising.csv", index_col=0)
df.head()


,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

X, y = df.drop("Sales", axis = "columns"), df["Sales"]

# feature engineered many more features, degree 3 -> 19 features
model_polynomial = PolynomialFeatures(3, include_bias=False)
polynomial_features = model_polynomial.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(polynomial_features, y, test_size=.33, random_state=42)

# x1, x1^2,x1^3, x1x2x3, x1^2x2^2x3^2 ....
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((134, 19), (66, 19), (134,), (66,))

## Feature standardization

In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# calculates parameters sigma and mu based on X_train and transform X_train
scaled_X_train = scaler.fit_transform(X_train) 

# uses the parameters sigma and mu that was calculated before to transform X_test
scaled_X_test = scaler.transform(X_test)

scaled_X_test.mean(), scaled_X_test.std(), scaled_X_train.mean(), scaled_X_train.std(), 

(-0.11982457640326809, 1.1245966534380971, -3.34898382919136e-17, 1.0)

## Regularizations

### Ridge regression (Tikhonov regularization) (L2-regularization)

In [21]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

def ridge_regression(X_test, penalty = 0):
    # alpha in Ridge is same as lambda in theory
    model_ridge = Ridge(alpha=penalty)
    model_ridge.fit(scaled_X_train, y_train)
    y_pred = model_ridge.predict(X_test)
    return y_pred

# ridge regression with penalty 0 is polynomial regression
y_pred = ridge_regression(scaled_X_test, penalty = 0)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE

(0.3748516441217865, 0.2650465950553601, 0.5148267621786576)

In [22]:
from sklearn.linear_model import LinearRegression

# Polynmomial regression 
model_linear = LinearRegression()
model_linear.fit(scaled_X_train, y_train)
y_pred = model_linear.predict(scaled_X_test)

MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE


(0.3748516441217832, 0.26504659505536404, 0.5148267621786614)

In [23]:
# ridge regression with penalty 0 is polynomial regression
y_pred = ridge_regression(scaled_X_test, penalty = 0.01)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE

(0.3716840615001231, 0.26202411194681025, 0.5118829084339603)

## LASSO - L1 regularization

In [26]:
from sklearn.linear_model import LassoCV

# cv is k, k -fold
model_lassoCV = LassoCV(n_alphas = 200, cv = 5) 
model_lassoCV.fit(scaled_X_train, y_train)



c:\Users\kokch\.virtualenvs\Machine-learning-AI22-eHcpWP3a\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.548e-01, tolerance: 3.571e-01
  model = cd_fast.enet_coordinate_descent(


LassoCV(cv=5, n_alphas=200)

In [28]:
# penalty found through 5-fold cross-validation
model_lassoCV.alpha_

0.004968802520343366

In [29]:
model_lassoCV.coef_

array([ 5.11468536,  0.42127203,  0.28896055, -4.63391705,  3.48972093,
       -0.390611  ,  0.        ,  0.        ,  0.        ,  1.24969939,
       -0.        ,  0.        ,  0.13795383, -0.01666923,  0.        ,
        0.        ,  0.10974819,  0.        ,  0.0458376 ])

In [31]:
y_pred = model_lassoCV.predict(scaled_X_test)

MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)

MAE, MSE, RMSE

(0.46802072322691207, 0.3410150044071009, 0.5839648999786724)

## ElasticNet

In [38]:
from sklearn.linear_model import ElasticNetCV

model_elastic = ElasticNetCV(l1_ratio=[0.01, .1,.5,.7,.8,.9,.95,1])
model_elastic.fit(scaled_X_train, y_train)


c:\Users\kokch\.virtualenvs\Machine-learning-AI22-eHcpWP3a\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.548e-01, tolerance: 3.571e-01
  model = cd_fast.enet_coordinate_descent(


ElasticNetCV(l1_ratio=[0.01, 0.1, 0.5, 0.7, 0.8, 0.9, 0.95, 1])

In [39]:
model_elastic.l1_ratio_

1.0

In [40]:
model_elastic.alpha_

0.004968802520343366